In [1]:
from infinipy.stateblock import StateBlock
from infinipy.statement import Statement, CompositeStatement, RelationalStatement, CompositeRelationalStatement
from infinipy.affordance import Affordance
from infinipy.transformer import Transformer, CompositeTransformer, RelationalTransformer, CompositeRelationalTransformer
import random
import time
from infinipy.content.statements import bigger_than, equals_to, has_attribute, is_true
from infinipy.gridmap import GridMap
from typing import List, Tuple, Optional

In [2]:
class CharacterBlock(StateBlock):
    def __init__(self, *args, can_store=True, can_move=True, can_be_stored=False, can_act=True, can_be_moved=True, **kwargs):
        super().__init__(*args, can_store=can_store, can_move=can_move, can_be_stored=can_be_stored, can_act=can_act, can_be_moved=can_be_moved, **kwargs)

class FloorBlock(StateBlock):
    def __init__(self, *args, can_store=False, can_move=False, can_be_stored=False, can_act=False, can_be_moved=False, **kwargs):
        super().__init__(*args, can_store=can_store, can_move=can_move, can_be_stored=can_be_stored, can_act=can_act, can_be_moved=can_be_moved, **kwargs)
        self.can_be_walked_on = True
class WallBlock(StateBlock):
    def __init__(self, *args, can_store=False, can_move=False, can_be_stored=False, can_act=False, can_be_moved=False, **kwargs):
        super().__init__(*args, can_store=can_store, can_move=can_move, can_be_stored=can_be_stored, can_act=can_act, can_be_moved=can_be_moved, **kwargs)
        
class TreasureBlock(StateBlock):
    def __init__(self, *args, can_store=False, can_move=False, can_be_stored=True, can_act=False, can_be_moved=False, **kwargs):
        super().__init__(*args, can_store=can_store, can_move=can_move, can_be_stored=can_be_stored, can_act=can_act, can_be_moved=can_be_moved, **kwargs)

def create_wall(x: int, y: int):
    return WallBlock(id=f"wall_{x}_{y}", owner_id="environment", name="Wall", position=(x, y, 0),
                                       reach=0, hitpoints=100, size="medium", blocks_move=True, blocks_los=True)
def create_floor(x: int, y: int):
    return FloorBlock(id=f"floor_{x}_{y}", owner_id="environment", name="Floor", position=(x, y, 0),
                                       reach=0, hitpoints=100, size="medium", blocks_move=False, blocks_los=False)

In [3]:
## movement logic 
def euclidean_distance(source: StateBlock, target: StateBlock) -> bool:
    source_position = source.position
    target_position = target.position
    dx = source_position[0] - target_position[0]
    dy = source_position[1] - target_position[1]
    distance = (dx**2 + dy**2)**0.5
    return distance <= source.reach

within_reach_statement = RelationalStatement(
    "Within Reach",
    "Checks if the target is within reach of the source based on their positions",
    euclidean_distance
)
has_walk_on_attribute = has_attribute("can_be_walked_on")
can_be_walked_on= is_true("can_be_walked_on")

def move(source: StateBlock, target: StateBlock):
    source.position = target.position

movement_transformer = RelationalTransformer("Movement Transformation", move)

move_to_affordance = Affordance(
    name="MoveTo",
    prerequisites=[(has_walk_on_attribute, 'target'),(can_be_walked_on, 'target'),(within_reach_statement, 'source')],
    consequences=[(movement_transformer, 'source')]
)

In [4]:
## pick up logic
# #source conditions
def has_inventory_space(source: StateBlock):
    if len(source.inventory) < source.inventory_size:
        return True
    return False

has_inventory_statement = Statement("Has Inventory Space", list, "Checks if the source has inventory space", has_inventory_space)
has_inventory_attribute = has_attribute("inventory")

can_store = CompositeStatement([(has_inventory_attribute, 'AND'), (has_inventory_statement, 'AND')])
#target conditions
is_pickable = CompositeStatement([(has_attribute("can_be_stored"), 'AND'), (is_true("can_be_stored"), 'AND')])
#effect definition
def pick_up(source: StateBlock, target: StateBlock):
    source.add_to_inventory(target)

pick_up_transformer = RelationalTransformer("Pick Up Transformation", pick_up)

pick_up_affordance = Affordance(
    name="PickUp",
    prerequisites=[(is_pickable, 'target'), (can_store, 'source'),
                   (within_reach_statement, 'source')],
    consequences=[(pick_up_transformer, 'source')]
)


In [5]:
def display_grid_with_gridmap(grid_map: GridMap):
    # Priority order for displaying entities (highest priority first)
    entity_priority = [CharacterBlock, TreasureBlock, WallBlock, FloorBlock]

    map_size = grid_map.map_size[0]
    grid = [[' ' for _ in range(map_size)] for _ in range(map_size)]
    
    for position, entities in grid_map.entities.items():
        x, y, _ = position

        # Determine which entity to display based on priority
        display_entity = None
        for priority_entity in entity_priority:
            for entity in entities:
                if isinstance(entity, priority_entity):
                    display_entity = entity
                    break
            if display_entity:
                break

        # Update the grid based on the entity to display
        if display_entity:
            if isinstance(display_entity, FloorBlock):
                grid[y][x] = '.'
            elif isinstance(display_entity, WallBlock):
                grid[y][x] = '#'
            elif isinstance(display_entity, TreasureBlock):
                print("found a tresure at", x, y)
                grid[y][x] = 'T'
            elif isinstance(display_entity, CharacterBlock):
                grid[y][x] = 'C'

    # Print the grid
    for row in grid:
        print(' '.join(row))

    # Print summary of character inventory
    for entity in grid_map.get_all_entities():  # Assuming get_all_entities() is a method to get all entities
        if isinstance(entity, CharacterBlock):
            print(f"{entity.name} has {len(entity.inventory)} items in inventory")


In [6]:
def create_square_room(size: Tuple[int,int], wall_thickness: int) -> GridMap:
    entities = []
    #door is top down corner and then up by one
    door_position = (size[0]-1,size[1]-2)
    vertical_wall_x = int(size[1]/2)
    vertical_wall_y = int(size[0] /2)
    vertical_wall_door_position = (vertical_wall_y, vertical_wall_x-1)
    door_positions = [door_position, vertical_wall_door_position]
    for x in range(size[0]):
        for y in range(size[1]):

            if (x,y) ==  door_position:
                entities.append(create_floor(x,y))
                print("left open for door at: ", (x,y))
            elif (x,y) == vertical_wall_door_position:
                entities.append(create_floor(x,y))
                print("left open for door at: ", (x,y))
            elif x < wall_thickness or x >= size[0] - wall_thickness or y < wall_thickness or y >= size[1] - wall_thickness:
                entities.append(create_wall(x,y))
            elif x == vertical_wall_x and y <= vertical_wall_y:
                entities.append(create_wall(x,y))
            elif y == vertical_wall_y and x <= vertical_wall_x:
                entities.append(create_wall(x,y))
            else:
                entities.append(create_floor(x,y))
    return entities

In [7]:
#test walk affordance outside of gridmap

char_0= CharacterBlock(id="char0", owner_id="player", name="Character0", position=(1, 1, 0),
                             reach=1, hitpoints=100, size="small", blocks_move=False, 
                             blocks_los=False, can_store=True, can_move=True, inventory_size=10)

floor_0 = create_floor (1,2)
print("char0 position", char_0.position)
print("floor0 position", floor_0.position)
move_to_affordance.apply(source_block=char_0, target_block=floor_0)
print("char0 position", char_0.position)

char0 position (1, 1, 0)
floor0 position (1, 2, 0)
char0 position (1, 2, 0)


In [8]:
#create a gridmap and entities

gridmap = GridMap((10,10))
char = CharacterBlock(id="char1", owner_id="player", name="Character1", position=(1, 1, 0),
                             reach=1, hitpoints=100, size="small", blocks_move=False, 
                             blocks_los=False, can_store=True, can_move=True, inventory_size=10)
treasure1 = TreasureBlock(id="treasure_1", owner_id="environment", name="Treasure1", position=(1, 2, 0),
                               reach=0, hitpoints=10, size="small", blocks_move=False, blocks_los=False, can_be_stored=True,inventory_size=0)

treasure2 = TreasureBlock(id="treasure_2", owner_id="environment", name="Treasure2", position=(1, 3, 0),
                               reach=0, hitpoints=10, size="small", blocks_move=False, blocks_los=False, can_be_stored=True,inventory_size=0)
gridmap.add_entity(char,char.position)
gridmap.print_entities()
gridmap.add_entity(treasure1,treasure1.position)
gridmap.print_entities()

char2 = CharacterBlock(id="char2", owner_id="player", name="Character2", position=(1, 4, 0),
                             reach=1, hitpoints=100, size="small", blocks_move=False, 
                             blocks_los=False, can_store=True, can_move=True, inventory_size=10)
room = create_square_room((10,10),1)
for entity in room:
    gridmap.add_entity(entity,entity.position)

display_grid_with_gridmap(gridmap)
print(len(gridmap.entities[(1,1,0)]),gridmap.entities[(1,1,0)])
print(len(gridmap.entities[(1,2,0)]),gridmap.entities[(1,2,0)])

# gridmap.print_entities()
print("execute affordance", pick_up_affordance.name)
gridmap.execute_affordance(pick_up_affordance, char, treasure1)
print(len(gridmap.entities[(1,1,0)]))
for entity in gridmap.entities[(1,1,0)]:
    print(entity.name)
print(len(gridmap.entities[(1,2,0)]))
for entity in gridmap.entities[(1,2,0)]:
    print(entity.name)
display_grid_with_gridmap(gridmap)
print(treasure2.position)
pick_up_affordance.apply(char2, treasure2)
print(treasure2.position)
char2.inventory[0].stored_in
gridmap.add_entity(char2,char2.position)
display_grid_with_gridmap(gridmap)


Printing entities for gridmap: gridmap_2023-11-20 00:35:38
Time since creation: 0.0
Character1 (1, 1, 0)
Printing entities for gridmap: gridmap_2023-11-20 00:35:38
Time since creation: 0.0004990100860595703
Character1 (1, 1, 0)
Treasure1 (1, 2, 0)
left open for door at:  (5, 4)
left open for door at:  (9, 8)
found a tresure at 1 2
# # # # # # # # # #
# C . . . # . . . #
# T . . . # . . . #
# . . . . # . . . #
# . . . . . . . . #
# # # # # # . . . #
# . . . . . . . . #
# . . . . . . . . #
# . . . . . . . . .
# # # # # # # # # #
Character1 has 0 items in inventory
2 [CharacterBlock(id='c9bfd46b-3be0-4d7e-a924-efd5c2648e78', owner_id='player', name='Character1', reach=1, hitpoints=100, size='small', blocks_move=False, blocks_los=False, can_store=True, can_be_stored=False, can_act=True, can_move=True, can_be_moved=True, position=(1, 1, 0), inventory=[], inventory_size=10, stored_in=None), FloorBlock(id='37fa156e-365b-4d67-a9b4-b6b6c0fdc29a', owner_id='environment', name='Floor', reach=0, h

In [9]:
#apply movment to char 1
canwalk, walkable_target = gridmap.affordance_applicable_at_position(move_to_affordance,char,(1,2,0))
print(walkable_target)
gridmap.execute_affordance(move_to_affordance, char, walkable_target)
display_grid_with_gridmap(gridmap)


FloorBlock(id='813b6497-1ba7-441f-ac91-cb8429e02a32', owner_id='environment', name='Floor', reach=0, hitpoints=100, size='medium', blocks_move=False, blocks_los=False, can_store=False, can_be_stored=False, can_act=False, can_move=False, can_be_moved=False, position=(1, 2, 0), inventory=[], inventory_size=10, stored_in=None)
Source position changed
Removing CharacterBlock(id='c9bfd46b-3be0-4d7e-a924-efd5c2648e78', owner_id='player', name='Character1', reach=1, hitpoints=100, size='small', blocks_move=False, blocks_los=False, can_store=True, can_be_stored=False, can_act=True, can_move=True, can_be_moved=True, position=(1, 2, 0), inventory=[TreasureBlock(id='74683c8f-b94a-4edc-b69f-96dcc3a71e33', owner_id='environment', name='Treasure1', reach=0, hitpoints=10, size='small', blocks_move=False, blocks_los=False, can_store=False, can_be_stored=True, can_act=False, can_move=False, can_be_moved=False, position=(1, 2, 0), inventory=[], inventory_size=0, stored_in=...)], inventory_size=10, store

In [10]:
def display_grid_with_lightmap(grid_map: GridMap, light_map: list, source: Optional[tuple] = None):
    # Priority order for displaying entities (highest priority first)
    entity_priority = [CharacterBlock, TreasureBlock, WallBlock, FloorBlock]

    map_size = grid_map.map_size[0]
    grid = [[' ' for _ in range(map_size)] for _ in range(map_size)]
    
    for position, entities in grid_map.entities.items():
        x, y, _ = position

        # Determine which entity to display based on priority
        display_entity = None
        for priority_entity in entity_priority:
            for entity in entities:
                if isinstance(entity, priority_entity):
                    display_entity = entity
                    break
            if display_entity:
                break

        # Update the grid based on the entity to display
        if display_entity:
            if isinstance(display_entity, FloorBlock):
                grid[y][x] = '.'
            elif isinstance(display_entity, WallBlock):
                grid[y][x] = '#'
            elif isinstance(display_entity, TreasureBlock):
                print("found a tresure at", x, y)
                grid[y][x] = 'T'
            elif isinstance(display_entity, CharacterBlock):
                grid[y][x] = 'C'
    if source is not None:
        source = source
    else:
        source = light_map[0]
    x0,y0,z0 = source
    grid[y0][x0] = 'X'
    for x,y,z in light_map:
        if (x,y,z) != source:
            grid[y][x] = 'L'
    # Print the grid
    for row in grid:
        print(' '.join(row))

    

In [11]:
light=gridmap.cast_light((1,2,0),5,70)
print(light)
display_grid_with_lightmap(gridmap,light)

[(1, 2, 0), (1, 3, 0), (2, 3, 0), (2, 4, 0), (3, 4, 0), (3, 5, 0)]
# # # # # # # # # #
# . . . . # . . . #
# X . . . # . . . #
# L L . . # . . . #
# C L L . . . . . #
# # # L # # . . . #
# . . . . . . . . #
# . . . . . . . . #
# . . . . . . . . .
# # # # # # # # # #


In [12]:
line = gridmap.line((1,2,0),(3,8,0))
print(line)
display_grid_with_lightmap(gridmap,line)

[(1, 2, 0), (1, 3, 0), (1, 4, 0), (2, 4, 0), (2, 5, 0)]
# # # # # # # # # #
# . . . . # . . . #
# X . . . # . . . #
# L . . . # . . . #
# L L . . . . . . #
# # L # # # . . . #
# . . . . . . . . #
# . . . . . . . . #
# . . . . . . . . .
# # # # # # # # # #


In [33]:
los, line_los = gridmap.line_of_sight((1,2,0),(7,5,0))
print(line_los)
display_grid_with_lightmap(gridmap,line_los,(1,2,0))

[(2, 2, 0), (2, 3, 0), (3, 3, 0), (4, 3, 0), (4, 4, 0), (5, 4, 0), (6, 4, 0), (6, 5, 0), (7, 5, 0)]
# # # # # # # # # #
# . . . . # . . . #
# X L . . # . . . #
# . L L L # . . . #
# C . . L L L . . #
# # # # # # L L . #
# . . . . . . . . #
# . . . . . . . . #
# . . . . . . . . .
# # # # # # # # # #


In [14]:
los = gridmap.shadow_casting((1,2,0))
print(los)
display_grid_with_lightmap(gridmap,los,(1,2,0))

[(8, 6, 0), (0, 1, 0), (9, 6, 0), (3, 5, 0), (2, 1, 0), (0, 3, 0), (4, 5, 0), (5, 5, 0), (1, 3, 0), (2, 3, 0), (6, 4, 0), (1, 1, 0), (3, 0, 0), (7, 4, 0), (1, 5, 0), (4, 0, 0), (4, 2, 0), (2, 5, 0), (4, 4, 0), (5, 2, 0), (8, 5, 0), (3, 2, 0), (9, 5, 0), (3, 4, 0), (0, 2, 0), (0, 4, 0), (1, 0, 0), (5, 4, 0), (2, 0, 0), (2, 2, 0), (6, 5, 0), (1, 2, 0), (3, 1, 0), (7, 5, 0), (1, 4, 0), (3, 3, 0), (4, 1, 0), (2, 4, 0), (4, 3, 0), (5, 1, 0), (5, 3, 0)]
# L L L L # # # # #
L L L L L L . . . #
L X L L L L . . . #
L L L L L L . . . #
L L L L L L L L . #
# L L L L L L L L L
# . . . . . . . L L
# . . . . . . . . #
# . . . . . . . . .
# # # # # # # # # #


In [15]:
path = gridmap.a_star((1,2,0),(3,8,0))
print(path)
display_grid_with_lightmap(gridmap,path,(1,2,0))

[(2, 3, 0), (3, 4, 0), (4, 4, 0), (5, 4, 0), (6, 5, 0), (5, 6, 0), (4, 7, 0), (3, 8, 0)]
# # # # # # # # # #
# . . . . # . . . #
# X . . . # . . . #
# . L . . # . . . #
# C . L L L . . . #
# # # # # # L . . #
# . . . . L . . . #
# . . . L . . . . #
# . . L . . . . . .
# # # # # # # # # #


In [16]:
path = gridmap.dijkstra((1,2,0),5)
path = list(path.keys())
print(path)
display_grid_with_lightmap(gridmap,path)

[(1, 2, 0), (2, 2, 0), (1, 1, 0), (1, 3, 0), (2, 1, 0), (2, 3, 0), (1, 4, 0), (2, 4, 0), (3, 1, 0), (3, 2, 0), (3, 3, 0), (3, 4, 0), (4, 1, 0), (4, 2, 0), (4, 3, 0), (4, 4, 0), (5, 4, 0), (6, 4, 0), (6, 3, 0), (6, 5, 0), (7, 3, 0), (6, 2, 0), (7, 2, 0), (7, 4, 0), (7, 5, 0), (6, 6, 0), (5, 6, 0), (7, 6, 0)]
# # # # # # # # # #
# L L L L # . . . #
# X L L L # L L . #
# L L L L # L L . #
# L L L L L L L . #
# # # # # # L L . #
# . . . . L L L . #
# . . . . . . . . #
# . . . . . . . . .
# # # # # # # # # #
